# Using EDHEC Risk Kit Part 5
### Explore use of the library to:
- Use the CAPM to determine the an appropriate return of an asset
- Explore the coefficients in CAPM
- Conduct factor analysis

### CAPM
- The expected return on any asset $i$ is the risk-free interest rate, $R_F$, plus a risk premium, which is the asset's market beta, $\beta_{iM}$, times the premium per unit of beta risk, $E(R_M) - R_f$
- $E(R_i) = R_f + \beta_{iM}(E(R_m) - R_f)$
    - Which can be rewritten to be expressed in terms of excess return of the asset
        - $E(R_i) - R_f = \beta_{iM}(E(R_m) - R_f)$
    - $\beta_{iM}$ for asset $i$ is the covariance of its return with the market return divided by the variance of the market return
    - (Market Beta) $\beta_{iM} = \frac{\text{cov}(R_i,R_M)}{\sigma^2(R_M)}$
    - Market beta is a risk premium
    - Assets with a market beta of zero must have an expected return equal to the risk free rate because they doesn't contribute to the variance in the market return

### CAPM assumptions
- Expected returns for each asset is linearly related to their market betas
- Expected return on market portfolio > expected return on assets with market beta of 0
    - Market betas for all assets are positive
- Assets uncorreleated with the market have expected returns equal $R_f$
- The beta premium is equal to $E(R_m) - R_f$

### Testing CAPM on market data
- Regress average asset returns on estimates of asset betas
- Model predicts the intercept is the riskfree rate
- Coefficient on beta is $E(R_M)-R_f$

### Issues that arise
- Tough to estimate market beta for an individual asset, can vary depending on time range
    - Estimate beta for portfolios of assets instead
- Residuals from the model for one asset are not independent from another's residuals
    - Caused by variation due to an attribute such as industry or location

In [1]:
import pandas as pd
import edhec_risk_kit as erk
import numpy as np
import statsmodels.api as sm

In [2]:
ind_rets = erk.get_ind_file('returns', n_inds=49)
fff = erk.get_fff_returns()

### What is the market beta for Healthcare industry stocks from 1970 to 2015?

In [3]:
hlth_excess = ind_rets.loc['1970':'2015', ['Hlth']] - fff.loc['1970':'2015', ['RF']].values
mkt_excess = fff.loc['1970':'2015', ['Mkt-RF']]
exp_var = mkt_excess.copy()
exp_var['Constant'] = 1
lm = sm.OLS(hlth_excess, exp_var).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Hlth   R-squared:                       0.390
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     352.2
Date:                Fri, 13 Mar 2020   Prob (F-statistic):           4.27e-61
Time:                        15:18:56   Log-Likelihood:                 732.28
No. Observations:                 552   AIC:                            -1461.
Df Residuals:                     550   BIC:                            -1452.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.1222      0.060     18.766      0.000       1.005       1.240
Constant   -6.651e-06      0.003     -0.002      0.998      -0.005       0.005
==============================================================================
Omnibus:                       49.539   Durbin-Watson:                   1.830
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              255.035
Skew:                          -0.107   Prob(JB):                     4.17e-56
Kurtosis:                       6.323   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### The market beta for healthcare stocks is the coefficient on the Mkt-RF variable in the model. In this case, it is 1.1222. This tells us that the expected excess return for healthcare stocks is 1.1222 times the excess return of the market

### Which industry had the highest market beta from 1960 to 2015?
- Consider all 49 industries

In [4]:
industries = list(ind_rets.columns)
betas = []
for ind in industries:
    ind_excess = ind_rets.loc['1970':'2015 ', [ind]] - fff.loc['1970':'2015', ['RF']].values
    lm = sm.OLS(ind_excess, exp_var).fit()
    betas.append(lm.params['Mkt-RF'])

beta_df = pd.DataFrame({'Beta':betas},
                        index = industries)

In [5]:
print(beta_df['Beta'].idxmax(), 'had a market beta of', np.round(beta_df['Beta'].max(),2))

Softw had a market beta of 1.59


### Lowest market beta?

In [6]:
print(beta_df['Beta'].idxmin(), 'had a market beta of', np.round(beta_df['Beta'].min(),2))

Util had a market beta of 0.52
